In [2]:
import pandas as pd
import os
import src.Common.Utils.Config.ConfigHelper as ConfigHelper
import shutil
import src.Common.EpisodeReplay.EpisodeReplay as EpisodeReplay

In [41]:
runGroup = "11"
envName = "FrozenLake"
maxReviewsPerAgent = 20

behaviouralType = "Human"
# behaviouralType = "Curated"

In [42]:
# create list of replays to review


# load 
aiReplaysPath = os.path.join("Data", envName, runGroup, f"{behaviouralType}_Episodes.json")
aiReplays =  ConfigHelper.LoadConfig(aiReplaysPath)

behaviourReplaysPath = os.path.join("Data", envName, "dev", "replays", behaviouralType, "stats.tsv")
behaviourReplays = pd.read_csv(behaviourReplaysPath, sep="\t")


In [32]:
# copy behaviourReplays to runGroup folder
fromPath = os.path.join("Data", envName, "dev", "replays", behaviouralType)
toPath = os.path.join("Data", envName, runGroup, "replays", behaviouralType)

if os.path.exists(toPath):
	shutil.rmtree(toPath)
shutil.copytree(fromPath, toPath)

'Data\\FrozenLake\\11\\replays\\Human'

In [33]:
# Create table of replay id, agent type, and column for score
# remove replays that can't be loaded
def LoadReplay(envName, runGroup, agentType, episodeId):
	
	path = os.path.join("Data", envName, runGroup, "replays", agentType, episodeId)
	try:
		replay = EpisodeReplay.EpisodeReplay.LoadFromFolder(path)
		return replay
	except:
		return None

replaysToReview = pd.DataFrame(columns=["ReplayId", "AgentType", "AgentId", "Predicted"])

for agentId, replays in aiReplays.items():
	
	# check it ends with behaviouralType
	if not agentId.endswith(behaviouralType):
		continue

	agentType = agentId.split("_")[0]


	for i in range(min(len(replays), maxReviewsPerAgent)):
		if LoadReplay(envName, runGroup, agentType, replays[i]) is None:
			continue

		replaysToReview = pd.concat([replaysToReview, pd.DataFrame({"ReplayId": [replays[i]], "AgentType": [agentType], "AgentId": [agentId], "Predicted": [None]})], ignore_index=True)

for i in range(min(len(behaviourReplays), maxReviewsPerAgent)):
	replay = behaviourReplays.iloc[i]
	replayId = replay.EpisodeId

	if LoadReplay(envName, runGroup, behaviouralType, replayId) is None:
		continue

	replaysToReview = pd.concat([replaysToReview, pd.DataFrame({"ReplayId": [replayId], "AgentType": ["Human"], "AgentId": ["Human"], "Predicted": [None]})], ignore_index=True)


In [34]:
# shuffle the order of the rows but keep all the data in each row together
replaysToReview = replaysToReview.sample(frac=1)

In [35]:
# loop through all replays and check if they can be loaded
for i in range(len(replaysToReview)):
	replay = replaysToReview.iloc[i]
	replayId = replay.ReplayId
	agentType = replay.AgentType

	if LoadReplay(envName, runGroup, agentType, replayId) is None:
		print(f"Can't load replay {replayId} for agent {agentType}")

In [36]:
# # save the df
# replaysToReviewPath = os.path.join("Data", envName, runGroup, "ReplaysToReview.tsv")
# replaysToReview.to_csv(replaysToReviewPath, sep="\t", index=False)
# save as json
replaysToReviewPath = os.path.join("Data", envName, runGroup, "ReplaysToReview.json")
replaysToReview.to_json(replaysToReviewPath, orient="records")

## Formate the results of the review

In [38]:
behaviourReviewResults = pd.read_json(replaysToReviewPath)
grouped = behaviourReviewResults.groupby("AgentType").aggregate({"Predicted": ["count", "sum"]})


grouped["Percent"] = grouped["Predicted"]["sum"] / grouped["Predicted"]["count"]
grouped["Norm_Percent"] = grouped["Percent"] / grouped["Percent"]["Human"]

# format the percent columns
grouped["Percent"] = grouped["Percent"].apply(lambda x: f"{x:.0%}")
grouped["Norm_Percent"] = grouped["Norm_Percent"].apply(lambda x: f"{x:.0%}")

grouped

Predicted     Percent Norm_Percent
              count sum                     
AgentType                                   
HardCoded        20  20    100%         133%
Human            20  15     75%         100%
ML               40  40    100%         133%
Random           20   0      0%           0%

## Create Tables of agents' results

In [25]:
envs = ["FrozenLake", "CartPole"]
agents = ["Human", "Random", "HardCoded", "ML"]

In [26]:
def LoadStats(envName, runGroup, agent):
	path = os.path.join("Data", envName, runGroup, "replays", agent, "stats.tsv")
	df = pd.read_csv(path, sep="\t")
	return df

In [27]:
df = LoadStats("FrozenLake", "6", "ML")
df

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\FrozenLake\\6\\replays\\ML\\stats.tsv'

In [ ]:
def EnvFitAgentSummary(summaries, df, agentName):
	mean = df["EpisodeTotalReward"].mean()
	sem = df["EpisodeTotalReward"].sem()

	summaries[f"{agentName} Mean"] = mean
	summaries[f"{agentName} sem"] = sem
	summaries[f"{agentName} n"] = len(df)

	return summaries

def GetEnvFitSummary(envName, runGroup):
	
	summaries = {}

	for agent in agents:
		try:
			df = LoadStats(envName, runGroup, agent)

			if agent == "ML":
				realSim = df.loc[df["UseRealSim"] == True]
				summaries = EnvFitAgentSummary(summaries, realSim, f"{agent}(Real Sim)")

				learnedSim = df.loc[df["UseRealSim"] == False]
				summaries = EnvFitAgentSummary(summaries, learnedSim, f"{agent}(Learned Model)")

			else:
				summaries = EnvFitAgentSummary(summaries, df, agent)
		except:
			pass

	# summaryDf = pd.DataFrame(summaries)
	return summaries

def GetEnvsSummary(runGroup):
	summaries = {}

	for env in envs:
		summaries[env] = GetEnvFitSummary(env, runGroup)

	df = pd.DataFrame(summaries)
	df = df.transpose()
	return df

df = GetEnvsSummary("6")
df


,Random Mean,Random sem,Random n,HardCoded Mean,HardCoded sem,HardCoded n,ML(Real Sim) Mean,ML(Real Sim) sem,ML(Real Sim) n,ML(Learned Model) Mean,ML(Learned Model) sem,ML(Learned Model) n
FrozenLake,0.004036,0.002015,991.0,1.0,0.0,990.0,0.719595,0.018478,592.0,1.0,0.0,2097.0
CartPole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,163.0,NaN,1.0


In [ ]:
df.round(3)

,Random Mean,Random sem,Random n,HardCoded Mean,HardCoded sem,HardCoded n,ML(Real Sim) Mean,ML(Real Sim) sem,ML(Real Sim) n,ML(Learned Model) Mean,ML(Learned Model) sem,ML(Learned Model) n
FrozenLake,0.004,0.002,991.0,1.0,0.0,990.0,0.72,0.018,592.0,1.0,0.0,2097.0
CartPole,nan,nan,nan,nan,nan,nan,nan,nan,0.0,163.0,nan,1.0
